In [4]:

import numpy as np
import pandas as pd
import networkx as nx
import pickle

df_nodes = pd.read_csv("/Users/garci061/Downloads/proximity.csv/nodes.csv")
df_nodes["program"] = df_nodes[" class"].str[:2]
df_edges1 = pd.read_csv("/Users/garci061/Downloads/proximity.csv/edges.csv")
df_edges2 = pd.read_csv("/Users/garci061/Downloads/diaries.csv/edges.csv")
df_edges3 = pd.read_csv("/Users/garci061/Downloads/survey.csv/edges.csv")
df_edges4 = pd.read_csv("/Users/garci061/Downloads/facebook.csv/edges.csv")
df_edges = pd.concat([df_edges1, df_edges2, df_edges3, df_edges4])
G_all = nx.from_pandas_edgelist(df_edges, source = "# source", target=" target")
pos = nx.spring_layout(G_all, seed = 1, scale=1)

for path in ["proximity", "diaries", "survey", "facebook"]:
    df_edges = pd.read_csv(f"/Users/garci061/Downloads/{path}.csv/edges.csv")
    G = nx.from_pandas_edgelist(df_edges, source = "# source", target=" target")
    nx.set_node_attributes(G, df_nodes.set_index("# index").to_dict()[" class"], "Classroom")
    nx.set_node_attributes(G, df_nodes.set_index("# index").to_dict()["program"], "Program")
    nx.set_node_attributes(G, df_nodes.set_index("# index").to_dict()[" gender"], "Gender")
    nx.write_graphml(G, f"data/{path}.graphml")
    
pickle.dump(pos, open("data/positions_nodes.pkl", "wb+"))

In [43]:
import networkx as nx
import pandas as pd
import hvplot.networkx as hvnx
import panel as pn
import numpy as np
import pickle
import requests
from io import BytesIO
import scipy

# Sample network file paths (you can replace these with your actual file paths)
network_files = {
    'Proximity': 'https://javier.science/panel_network/data/proximity.graphml',
    'Survey': 'https://javier.science/panel_network/data/survey.graphml',
    'Facebook': 'https://javier.science/panel_network/data/facebook.graphml',
    'Diaries': 'https://javier.science/panel_network/data/diaries.graphml'
}


def load_network(file_path):
    # Load network data from file
    return nx.read_graphml(path)

def compute_centrality_measures(G, measure):
    # Calculate centrality measures
    measures = {
        'Degree': nx.degree_centrality,
        'Betweenness': nx.betweenness_centrality,
        'Closeness': nx.closeness_centrality,
        'PageRank': nx.pagerank,
        # Add more centrality measures as needed
    }
    return measures[measure](G)

# Compute community detection using Louvain algorithm
def detect_communities(G):
    communities = nx.algorithms.community.louvain_communities(G, seed=42)
    partition = {node: cid for cid, comm in enumerate(communities) for node in comm}
    return partition


def display_statistics(G):
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    density = nx.density(G)
    transitivity = nx.transitivity(G)
    assortativity = nx.degree_assortativity_coefficient(G)
    gender_assort = nx.assortativity.attribute_assortativity_coefficient(G, "Gender")
    class_assort = nx.assortativity.attribute_assortativity_coefficient(G, "Classroom")
    program_assort = nx.assortativity.attribute_assortativity_coefficient(G, "Program")
    
    diameter = nx.diameter(G)
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    num_components = nx.number_connected_components(G)
    
    stats = f"Number of Nodes: {num_nodes}<br>"
    stats += f"Number of Edges: {num_edges}<br>"
    stats += f"Average Degree: {avg_degree:2.2f}<br>"
    stats += f"Diameter: {diameter}<br>"
    stats += f"Density: {density:2.2f}<br>"
    stats += f"Transitivity: {transitivity:2.2f}<br>"
    stats += f"Degree assortativity: {assortativity:2.2f}<br>"
    stats += f"Gender assortativity: {gender_assort:2.2f}<br>"
    stats += f"Clasroom assortativity: {class_assort:2.2f}<br>"
    stats += f"Program assortativity: {program_assort:2.2f}<br>"
    
    return pn.pane.HTML(stats)

# Visualize network
def visualize_network(G, centrality_measure="Degree", community_measure="Community (inferred)"):
    # Load node positions
    pos_response = requests.get("https://javier.science/panel_network/data/positions_nodes.pkl")
    pos = pickle.load(BytesIO(pos_response.content))
    
    # Compute centrality measures
    centrality = compute_centrality_measures(G, centrality_measure)
    
    # Compute community detection
    if community_measure=="Community (inferred)":
        communities = detect_communities(G)
    else:
        communities = nx.get_node_attributes(G, community_measure)

    communities = [communities[node] for node in G.nodes]
    cs = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    # Create a mapping dictionary
    mapping_dict = {com: index for index, com in enumerate(set(communities))}
    mapping_dict_r = {i: com for (com, i) in mapping_dict.items()}
    # Map letters to indexes using list comprehension
    indexes = [mapping_dict[com] for com in communities]
    color = [cs[i%10] for i in indexes]

    
    # Prepare size based on centrality
    size = np.array([centrality[node] for node in G.nodes()])
    size = (size-np.min(size))/(np.max(size)-np.min(size))
    size *= 100
    size += 10

    # Draw network
    spring = hvnx.draw(G, 
                       pos = {node: pos[int(node)] for node in G},
                       node_size=size, 
                       node_color=color, 
                       edge_color='darkgray', 
                       edge_alpha=0.2
                      )
    

    return spring
    


# Create file selector widget
file_selector = pn.widgets.Select(options=network_files, name='Select Network File')

# Create centrality measure selector widget
centrality_selector = pn.widgets.Select(options=['Degree', 'Betweenness', 'Closeness', 'PageRank'], name='Node size: Centrality Measure')

# Create community detection toggle
community_toggle = pn.widgets.Select(options=['Community (inferred)', 'Gender', 'Program', 'Classroom'], name='Node coor: Centrality Measure')

# Read network
response = requests.get(file_selector.value)
G = nx.read_graphml(BytesIO(response.content))
connected_components = list(nx.connected_components(G))

# Select the largest connected component
largest_component = max(connected_components, key=len)
G = G.subgraph(largest_component)
stats_table = display_statistics(G)

def update_graph(event):
    global G
    global stats_table
    # Read network
    response = requests.get(event.new)
    G = nx.read_graphml(BytesIO(response.content))
    connected_components = list(nx.connected_components(G))
    
    # Select the largest connected component
    largest_component = max(connected_components, key=len)
    G = G.subgraph(largest_component)
    stats_table = display_statistics(G)


# Create panel app layout
def update_app(file_path, centrality_measure, community_detection):
    global G
    global stats_table
    
    network_plot = visualize_network(G, centrality_measure, community_detection)
    
    return pn.Row(
        pn.Column(file_selector, centrality_selector, community_toggle, stats_table),
        pn.Column(network_plot)
    )

file_selector.param.watch(update_graph, "value")

app = pn.bind(update_app,
        file_path=file_selector.param.value, 
        centrality_measure=centrality_selector.param.value, 
        community_detection=community_toggle.param.value)



# Display the app
layout = pn.Row(app)
layout.show()

Launching server at http://localhost:58043


'https://javier.science/panel_network/data/diaries.graphml'

In [ ]:
#!panel serve app.py 

# Converting the app we made into a PWA using pyodide (the server becomes our browser)
!panel convert app.py --to pyodide-worker --out ./ --pwa 

Launching server at http://localhost:58069


In [11]:
# You'll need to run an http server to see the app (but hey, github pages has it!)
!python3 -m http.server

Serving HTTP on :: port 8000 (http://[::]:8000/) ...
::1 - - [26/Mar/2024 15:37:44] "GET / HTTP/1.1" 200 -
::1 - - [26/Mar/2024 15:37:45] "GET /app.js HTTP/1.1" 200 -
::1 - - [26/Mar/2024 15:37:45] code 404, message File not found
::1 - - [26/Mar/2024 15:37:45] "GET /favicon.ico HTTP/1.1" 404 -
::1 - - [26/Mar/2024 15:38:34] "GET / HTTP/1.1" 200 -
^C

Keyboard interrupt received, exiting.
